[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/10-ME/full_search_ME.ipynb)

# Full search block-based ME (Motion Estimation)
The predicted frame is divided into blocks and each one is characterized by a motion vector using exhaustive search. This guarantees reaching the global optimal (the best motion field).

In [ ]:
import numpy as np
!ln -sf ~/MRVC/src/image_3.py .
import image_3 as frame
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg as YUV
!ln -sf ~/MRVC/src/config.py .
!ln -sf ~/MRVC/src/debug.py .
!ln -sf ~/MRVC/src/motion.py .
import motion
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import cv2 as cv

In [ ]:
def normalize(img):
    max_component = np.max(img)
    min_component = np.min(img)
    max_min_component = max_component - min_component
    return (img-min_component)/max_min_component

def show_frame(frame, prefix=None):
    #frame = normalize(frame)
    plt.figure(figsize=(10,10))
    plt.title(prefix, fontsize=20)
    plt.imshow(frame)

def show_vectors2(flow, dpi=150, downsampling=1):
    #plt.figure.set_dpi(200)
    plt.figure(dpi=dpi)
    plt.quiver(flow[..., 0][::downsampling], flow[..., 1][::downsampling])
    plt.show()
    
def show_vectors(flow, dpi=150):
    #plt.figure.set_dpi(200)
    plt.figure(dpi=dpi)
    plt.quiver(flow[..., 0], flow[..., 1])
    plt.show()

In [ ]:
R = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 0).astype(np.int16))[...,0]
P = YUV.from_RGB(frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 1).astype(np.int16))[...,0]

In [ ]:
show_frame(R, "Reference")

In [ ]:
show_frame(P, "Predicted")

In [ ]:
P.max()

In [ ]:
P.min()

## Compute dense motion

In [ ]:
def full_search_block_based_ME(predicted, reference, block_side=16, search_range=32):
    extended_reference = np.zeros((reference.shape[0] + search_range, reference.shape[1] + search_range))
    extended_reference[search_range//2 : reference.shape[0] + search_range//2,
                       search_range//2 : reference.shape[1] + search_range//2] = reference
    flow = np.zeros((predicted.shape[0]//block_side, predicted.shape[1]//block_side, 2), dtype=np.int8)
    for block_y_coor in range(0, predicted.shape[0], block_side):
        for block_x_coor in range(0, predicted.shape[1], block_side):
            #print(block_y_coor, block_x_coor)
            errors_by_search_area = np.empty((search_range, search_range))
            counter = 0
            for search_range_y_coor in range(block_y_coor - search_range//2, block_y_coor + search_range//2):
                for search_range_x_coor in range(block_x_coor - search_range//2, block_x_coor + search_range//2):
                    #left_top_corner = [block_y_coor + search_range_y_coor, block_x_coor + search_range_x_coor]
                    #right_bottom_corner = [left_top_corner[0] + search_range, left_top_corner[1] + search_range]
                    #print(left_top_corner, right_bottom_corner)
                    print(search_range_y_coor, search_range_x_coor)
                    extended_reference_block = extended_reference[search_range_y_coor + search_range//2 :
                                                                  search_range_y_coor + search_range//2 + block_side,
                                                                  search_range_x_coor + search_range//2 :
                                                                  search_range_x_coor + search_range//2 + block_side]
                    predicted_block = predicted[block_y_coor :
                                                block_y_coor + block_side,
                                                block_x_coor :
                                                block_x_coor + block_side]
                    errors_in_search_area = extended_reference_block - predicted_block
                                            
                    #print("extended", extended_reference_block)
                    #print("predicted", predicted_block)
                    #print("error", errors_in_search_area)
                    absolute_error_by_block = np.sum(np.abs(errors_in_search_area))
                    #print(counter, absolute_error_by_block)
                    errors_by_search_area[(search_range_y_coor + search_range//2) % search_range,
                                          (search_range_x_coor + search_range//2) % search_range] = absolute_error_by_block
                    counter += 1
            #print(errors_by_search_area)
            mv_index = np.argmin(errors_by_search_area)
            #print(mv_index)
            MV_y = mv_index // block_side
            MV_x = mv_index % block_side
            flow[block_y_coor // block_side, block_x_coor // block_side] = (MV_y, MV_x)
    return flow
block_side = 4
flow = full_search_block_based_ME(P, R, block_side=block_side, search_range=8)

In [ ]:
flow.shape

In [ ]:
for y in range(P.shape[0]//block_side):
    for x in range(P.shape[1]//block_side):
        print(flow[y, x], end=' ')
    print()

In [ ]:
show_vectors(flow[::1, ::1])

In [ ]:
flow.max()

In [ ]:
flow.min()

In [ ]:
for i in range(64):
    for j in range(64):
        print((np.rint(flow)[i,j][1]), end=' ')
    print()

In [ ]:
show_frame(flow[...,0])

In [ ]:
show_frame(flow[...,1])

In [ ]:
show_frame(motion.colorize(flow), "Motion in HSV color domain")

In [ ]:
R = frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 0)
P = frame.read("/home/vruiz/MRVC/sequences/moving_circles/", 1)
prediction = motion.make_prediction(R, flow)
show_frame(prediction, "Prediction")
show_frame(P, "Predicted")

In [ ]:
show_frame(P-prediction, "Error")